In [1]:
import json
import torch
import re
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

In [2]:
generator = torch.Generator().manual_seed(420)

# N-Grams

## Prepare the dataset

In [3]:
# Specifies the train/val/test split 
#(test will be inferred; it is just the remaining data)

train_split = 0.8
val_split = 0.1 

In [4]:
with open('pokemons.json', 'r') as f:
    words = json.load(f)['names']

# remove weird characters from names
words = [re.sub(r'[^a-zA-Z]', '', word) for word in words]

# a character that serves as both start and end sentence token
special_token = "."

# prepare an alphabet: Set[str]
alphabet = list(set(''.join(words)))
alphabet.append(".")
alphabet.sort()

num_training_words = round(train_split * len(words))
num_validation_words = round(val_split * len(words))
num_test_words = len(words) - (num_training_words + num_validation_words)

training_words = words[:num_training_words]
validation_words = words[num_training_words:(num_validation_words + num_training_words)]
test_words = words[(num_validation_words + num_training_words):]

print(f"The dataset contains {len(words)} names.\n")

print(f"Choosing {len(training_words)} words for training.")
print(f"Choosing {len(validation_words)} words for validation.")
print(f"Choosing {len(test_words)} words for testing.\n")

print(f"Our alphabet consists of {len(alphabet)} characters: \n{alphabet}")

The dataset contains 908 names.

Choosing 726 words for training.
Choosing 91 words for validation.
Choosing 91 words for testing.

Our alphabet consists of 27 characters: 
['.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Define N-Gram Models

In [5]:
class NGramModel(ABC):
    def __init__(self, train, test, alphabet):
        self.train = train
        self.test = test 
        self.alphabet = alphabet
    
    @abstractmethod 
    def compute_counts(self):
        """
        Auxiliary method for compute_probabilities method that 
        creates count matrix
        """
        raise NotImplementedError()
        
    @abstractmethod
    def compute_probabilities(self):
        """
        Compute the NGram model from the training data
        """
        raise NotImplementedError()
      
    @abstractmethod
    def evaluate(self): 
        """
        Evaluate of test data
        """
        raise NotImplementedError()

In [6]:
class RandomModel(NGramModel):
    
    def __init__(self, train, test, alphabet):
        super().__init__(train, test, alphabet)
        self.counts = self.compute_counts
        self.probabilities = self.compute_probabilities()
        
    def compute_counts(self):
        return None
      
    def compute_probabilities(self):
        """
        Create the Model; an uniform matrix filled with probability values
        P(char_next|char_prev) = 1/len(self.alphabet)
        """
        return torch.full([len(self.alphabet), len(self.alphabet)], fill_value = 1/len(self.alphabet))
    
    def evaluate(self): 
        """
        Compute Negative Log Likelihood of the Model
        """
        log_likelihood = 0
        count_bigrams = 0
        for w in self.test:
            chs = ["."] + list(w) + ["."]
            for ch1, ch2 in zip(chs, chs[1:]):
                row_index, column_index = self.alphabet.index(ch1), self.alphabet.index(ch2)
                log_likelihood += torch.log(self.probabilities[row_index, column_index])
                count_bigrams += 1
        return -log_likelihood.item()/count_bigrams

In [7]:
class BigramModel(NGramModel):
    
    def __init__(self, train, test, alphabet, smoothing_counts = 0):
        super().__init__(train, test, alphabet)
        self.counts = self.compute_counts()
        self.probabilities = self.compute_probabilities(smoothing_counts = smoothing_counts)
    
    def compute_counts(self):
        counts = torch.zeros([len(self.alphabet), len(self.alphabet)], dtype=torch.int32)
        for w in self.train:
            chs = ["."] + list(w) + ["."]
            for ch1, ch2 in zip(chs, chs[1:]):
                row_index, column_index = self.alphabet.index(ch1), self.alphabet.index(ch2)
                counts[row_index, column_index] += 1
        return counts
    
    def compute_probabilities(self,smoothing_counts):
        # adding small number to avoid numerical instabilities
        counts = self.counts + 1e-15 + smoothing_counts
        probabilities = (counts)/torch.sum(counts, dim=1, keepdim=True)
        return probabilities
    
    def generate(self, generator):
        old_char, new_char = ".", None
        name = ""
        while new_char != special_token:
            probs = self.probabilities[self.alphabet.index(old_char)]
            sample=torch.multinomial(probs, num_samples=1, replacement=True, generator=generator)
            new_char = alphabet[sample.item()]
            name += new_char
            old_char = new_char
        return name[:-1] # removing last character, as it will be "."
    
    def evaluate(self): 
        """
        Compute Negative Log Likelihood of the Model
        """
        log_likelihood = 0
        count_bigrams = 0
        for w in self.test:
            chs = ["."] + list(w) + ["."]
            for ch1, ch2 in zip(chs, chs[1:]):
                row_index, column_index = self.alphabet.index(ch1), self.alphabet.index(ch2)
                log_likelihood += torch.log(self.probabilities[row_index, column_index])
                count_bigrams += 1
        return -log_likelihood.item()/count_bigrams
        
    def visualize_counts(self):
        plt.figure(figsize=(20,20))
        plt.imshow(self.counts)
        for i in range(len(self.alphabet)):
            for j in range(len(self.alphabet)):
                bigram_str = self.alphabet[i] + self.alphabet[j]
                plt.text(j, i, bigram_str, ha="center", va = "bottom", color="gray")
                plt.text(j, i, self.counts[i,j].item(), ha="center", va = "top", color="gray")
        plt.axis("off")

In [8]:
class TrigramModel(NGramModel):
    def __init__(self, train, test, alphabet, smoothing_counts = 0):
        
        super().__init__(train, test, alphabet)
        
        # because of the complexity of the problem, I am storing the mapping
        # from bigrams to counts matrix rows (as well as its inverse) in a dictionary
        self.bigram_to_index = self.compute_bigram_to_index_mapping()
        self.index_to_bigram = {v: k for k, v in self.bigram_to_index.items()}
        
        self.counts = self.compute_counts()
        self.probabilities = self.compute_probabilities(smoothing_counts = smoothing_counts)
        
    def compute_bigram_to_index_mapping(self):
        bigram_to_index = {}
        i = 0
        for char1 in alphabet:
            for char2 in alphabet:
                bigram_to_index[char1 + char2] = i
                i += 1
        return bigram_to_index

    
    def compute_counts(self):
        counts = torch.zeros([len(self.alphabet) * len(self.alphabet), len(self.alphabet)], dtype=torch.int32)
        for w in self.train:
            chs = ["."] + list(w) + ["."]
            for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
                row_index = self.bigram_to_index[ch1+ch2]
                column_index = self.alphabet.index(ch3)
                counts[row_index, column_index] += 1
        return counts
    
    def compute_probabilities(self,smoothing_counts):
        counts = self.counts + 1e-15 + smoothing_counts
        probabilities = (counts)/torch.sum(counts, dim=1, keepdim=True)
        # the hack below sets the probability of P(char_next="."|char_prev) = ".."
        # why would we like to sample an empty word?
        probabilities[0][0] = 0

        return probabilities
            
    
    def generate(self, generator):
        bigram, new_char = "..", None
        name = ""
        while new_char != ".":
            probs = self.probabilities[self.bigram_to_index[bigram]]
            sample=torch.multinomial(probs, num_samples=1, replacement=True, generator=generator)
            new_char = self.alphabet[sample.item()]  
            name += new_char
            bigram = bigram[1] + new_char
        return name[:-1]
        
    def evaluate(self): 
        log_likelihood = 0
        count_bigrams = 0
        for w in self.test:
            chs = ["."] + list(w) + ["."]
            for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
                row_index = self.bigram_to_index[ch1+ch2]
                column_index = self.alphabet.index(ch3)
                log_likelihood += torch.log(self.probabilities[row_index, column_index])
                count_bigrams += 1
        return -log_likelihood.item()/count_bigrams
    
    def visualize_counts(self, first_n_rows):
        plt.figure(figsize=(20,20))
        plt.imshow(self.counts[:first_n_rows,:])
        for i in range(first_n_rows):
            for j in range(len(alphabet)):
                bigram_str = self.index_to_bigram[i] + alphabet[j] 
                plt.text(j, i, bigram_str, ha="center", va = "bottom", color="gray")
                plt.text(j, i, self.counts[i,j].item(), ha="center", va = "top", color="gray")
        plt.axis("off")
        

In [9]:
def smoothing_parameter_tuning(n_gram_model_class, parameter_space = 5):
    """
    Hyperparameter tuning on validation set
    """
    nll, best_smoothing = 10000, None
    for smoothing in range(parameter_space):
        model = n_gram_model_class(train=training_words, test=validation_words, alphabet=alphabet, smoothing_counts=smoothing)
        print(f"Smoothing parameter: {smoothing}, NLL: {model.evaluate()}")
        if model.evaluate() < nll:
            nll, best_smoothing = model.evaluate(), smoothing
    return best_smoothing

## Evaluate N-Gram Models

In [10]:
best_smoothing_bigram = smoothing_parameter_tuning(BigramModel)
print('-----')
best_smoothing_trigram = smoothing_parameter_tuning(TrigramModel)

Smoothing parameter: 0, NLL: 3.092331807324841
Smoothing parameter: 1, NLL: 2.7274323870421973
Smoothing parameter: 2, NLL: 2.7469723452428343
Smoothing parameter: 3, NLL: 2.7670279533240447
Smoothing parameter: 4, NLL: 2.786065012937898
-----
Smoothing parameter: 0, NLL: 11.105617907961095
Smoothing parameter: 1, NLL: 2.836505812939031
Smoothing parameter: 2, NLL: 2.9233312249527197
Smoothing parameter: 3, NLL: 2.9821752718614913
Smoothing parameter: 4, NLL: 3.0241266519947767


In [11]:
"""
Evaluation on the test set
"""
random_model = RandomModel(train=training_words, test=test_words, alphabet=alphabet)
bigram_model = BigramModel(train=training_words, test=test_words, alphabet=alphabet, smoothing_counts=best_smoothing_bigram)
trigram_model = TrigramModel(train=training_words, test=test_words, alphabet=alphabet, smoothing_counts=best_smoothing_bigram)
print(f"The quality of the Trigram model (NLL): {trigram_model.evaluate()}")
print(f"The quality of the Bigram model (NLL): {bigram_model.evaluate()}")
print(f"The quality of the Random model (NLL): {random_model.evaluate()}")

The quality of the Trigram model (NLL): 2.8549500792176574
The quality of the Bigram model (NLL): 2.7148289077039394
The quality of the Random model (NLL): 3.295827860867711


In [12]:
# bigram_model.visualize_counts()
#trigram_model.visualize_counts(first_n_rows = 50)

In [13]:
# for generation, the models without the label smoothing are doing best
bigram_model = BigramModel(train=words, test=test_words, alphabet=alphabet)
trigram_model = TrigramModel(train=words, test=test_words, alphabet=alphabet)

bigram_names = [bigram_model.generate(generator=generator) for _ in range(10)]
print(f"Names generated by a bigram model:\n{bigram_names}")
print("----------------------------------")
trigram_names = [trigram_model.generate(generator=generator) for _ in range(10)]
print(f"Names generated by a bigram model:\n{trigram_names}")

Names generated by a bigram model:
['kos', 'aruir', 'thictamp', 'liolveadr', 'fehianflrilfeodilppicodstron', 'mopet', 'arras', 'kabeolitampilirisinopk', 'swlatened', 'finiostil']
----------------------------------
Names generated by a bigram model:
['fearsh', 'umer', 'yam', 'whinchi', 'esse', 'pa', 'hariza', 'blactilbatuff', 'ill', 'me']


### Additional excercises

**E01**: Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

**A:** Using a trigram model increases the number of parameters from $27 * 27$ to $27 * 27* 27$. Given the fact that our dataset is quite limited, I suspect that it is justified that using a model with too much capacity may worsen the results. So all in all, the trigram model does slightly worse then the bigram model, but still significantly better than the baseline (random model).

**E02**: Split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

**A**: This task has been done jointly with E01.

**E03**: Use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

**A**: Smoothing equal to one significantly improves the quality of the model over no smoothing - little more regularization of the model improves the generalization. However, further increasing the smoothing leads to increase of entropy and finally, rapid degradation of the model. Best loss with N-Gram models is achieved for Bigram model; I am pretty happy with it.

# MLP

## Prepare the dataset

In [14]:
def words_to_nn_dataset(words):
    inputs, labels = [], []
    for w in words:
        chs = ["."] + list(w) + ["."]
        for ch1, ch2 in zip(chs, chs[1:]):
            row_index, column_index = alphabet.index(ch1), alphabet.index(ch2)
            inputs.append(row_index)
            labels.append(column_index)
    inputs = torch.nn.functional.one_hot(torch.tensor(inputs), num_classes = len(alphabet)).float()
    labels = torch.tensor(labels)
    return (inputs, labels)

In [15]:
train_set = words_to_nn_dataset(training_words)
val_set = words_to_nn_dataset(validation_words)
test_set = words_to_nn_dataset(test_words)

In [16]:
class MLP:
    def __init__(self, train_data, val_data, test_data, generator, lr = 20, lambda_ = 0., use_cross_entropy = False): 
        self.train_inputs, self.train_labels = train_data
        self.val_inputs, self.val_labels = val_data
        self.test_inputs, self.test_labels = test_data
        self.lambda_ = lambda_
        self.lr = lr
        self.ce = use_cross_entropy
        self.W = torch.randn(len(alphabet), len(alphabet), requires_grad = True, generator=generator)
        
    def nll(self, output, inputs, labels):
        loss = -output[torch.arange(inputs.size()[0]), labels].log().mean()
        loss += self.lambda_ *(self.W**2).mean()
        return loss
    
    def nll_w_cross_entropy(self, logits, labels):
        loss = torch.nn.functional.cross_entropy(logits, labels)
        loss += self.lambda_ * (self.W**2).mean()
        return loss
        
    def train(self, epochs, verbose = False):
        for epoch in range(epochs):
            self.W.grad = None
            inputs = self.train_inputs
            logits = (inputs @ self.W)
            if self.ce:
                train_loss = self.nll_w_cross_entropy(logits, self.train_labels)
            else:
                softmax = logits.exp() 
                softmax = softmax/ softmax.sum(1, keepdims=True)
                train_loss = self.nll(softmax, self.train_inputs, self.train_labels)
            train_loss.backward()
            self.W.data += -self.lr * self.W.grad
            
            inputs = self.val_inputs
            logits = (inputs @ self.W)
            val_loss = self.nll_w_cross_entropy(logits, self.val_labels)
            if epoch % 100 == 0 and verbose:
                print(f"Epoch: {epoch} | Train loss: {train_loss} | Val loss: {val_loss}")
                
    def generate(self, generator):
        old_char, new_char = ".", None
        name = ""
        while new_char != special_token:
            logits = mlp.W[alphabet.index(old_char)].exp()
            pseudo_probs = logits / logits.sum()
            sample=torch.multinomial(pseudo_probs, num_samples=1, replacement=True, generator=generator)
            new_char = alphabet[sample.item()]  
            name += new_char
            old_char = new_char
        return name[:-1]
    
    def evaluate(self):
        inputs = self.test_inputs
        logits = (inputs @ self.W)
        test_loss = self.nll_w_cross_entropy(logits, self.test_labels)
        return test_loss
            

## Train

In [17]:
mlp = MLP(train_data = train_set, val_data = val_set, test_data = test_set, generator=generator)
mlp.train(epochs = 1001, verbose=True)

Epoch: 0 | Train loss: 3.701826333999634 | Val loss: 3.5069384574890137
Epoch: 100 | Train loss: 2.6354689598083496 | Val loss: 2.745215892791748
Epoch: 200 | Train loss: 2.601518154144287 | Val loss: 2.7292656898498535
Epoch: 300 | Train loss: 2.590712070465088 | Val loss: 2.726243019104004
Epoch: 400 | Train loss: 2.5855460166931152 | Val loss: 2.7253806591033936
Epoch: 500 | Train loss: 2.5824830532073975 | Val loss: 2.725008726119995
Epoch: 600 | Train loss: 2.5804200172424316 | Val loss: 2.724853515625
Epoch: 700 | Train loss: 2.5789215564727783 | Val loss: 2.724848508834839
Epoch: 800 | Train loss: 2.5777781009674072 | Val loss: 2.7249608039855957
Epoch: 900 | Train loss: 2.5768749713897705 | Val loss: 2.725160837173462
Epoch: 1000 | Train loss: 2.576143980026245 | Val loss: 2.7254269123077393


## Evaluate

In [18]:
# notice how similar it is to the Bigram model
print(f"The quality of the MLP (NLL): {mlp.evaluate()}")

The quality of the MLP (NLL): 2.7145590782165527


In [19]:
mlp = MLP(train_data = words_to_nn_dataset(words), val_data = val_set, test_data = test_set, generator=generator)
mlp.train(epochs = 1001)
mlp_names = [mlp.generate(generator=generator) for _ in range(10)]
print(f"Names generated by a bigram model:\n{bigram_names}")
# yay! same names as Bigram model! Andrej did not lie to us!

Names generated by a bigram model:
['kos', 'aruir', 'thictamp', 'liolveadr', 'fehianflrilfeodilppicodstron', 'mopet', 'arras', 'kabeolitampilirisinopk', 'swlatened', 'finiostil']


In [20]:
# E04 Answer
inputs = []
for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        row_index, _ = alphabet.index(ch1), alphabet.index(ch2)
        inputs.append(row_index)

W = torch.randn(len(alphabet), len(alphabet), requires_grad = True, generator=generator)
inputs_ = torch.nn.functional.one_hot(torch.tensor(inputs), num_classes = len(alphabet)).float()
assert [True for i in inputs_ @ W == W[inputs]]


### Additional excercises

**E04**: We saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

**A**: Look at the line of code marked as "E04 Answer"

**E05**: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

**A**: Few reasons. It has higher level of abstraction (easy to use it off-the-shelf), can compute NLL between two distributions, maybe computationally more efficient?